<a href="https://colab.research.google.com/github/softwareeng1/Acquiring-financial-data/blob/main/Dataprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
df = yf.download('AAPL',
                 start='2010-01-01',
                 end='2021-01-01',
                 progress=False)
#Keep adjusted close prices only
df = df.loc[ :,['Adj Close']]


In [3]:
#calculate simple and log returns
df['simple_rtrn'] = df['Adj Close'].pct_change()
df['log_rtrn'] = np.log(df['Adj Close']/df['Adj Close'].shift(1))
df = df.dropna()
df.head()


,Adj Close,simple_rtrn,log_rtrn
Date,,,
2010-01-05,6.473148,0.001729,0.001727
2010-01-06,6.370185,-0.015906,-0.016034
2010-01-07,6.358408,-0.001849,-0.001850
2010-01-08,6.400681,0.006648,0.006626
2010-01-11,6.344219,-0.008821,-0.008860


# Adjusting For Inflation

In [4]:
!pip install Nasdaq-Data-Link

In [5]:
import nasdaqdatalink
import pandas as pd


In [8]:
nasdaqdatalink.ApiConfig.api_key = 'place holder for api key'

In [16]:
df = df.resample('M').last()

In [18]:
df_cpi = nasdaqdatalink.get(
    dataset = "FRED/CPIAUCSL", # Changed the dataset code to a valid one
    start_date = '2010-01-01',
    end_date = '2021-01-01').rename(columns = {'Value':'cpi'})

In [20]:
df_cpi

,cpi
Date,
2010-01-01,217.488
2010-02-01,217.281
2010-03-01,217.353
2010-04-01,217.403
2010-05-01,217.290
...,...
2020-09-01,260.190
2020-10-01,260.352
2020-11-01,260.721


USING OTHER METHODS TO GET CPI DATA

In [34]:
!pip install cpi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.0/32.0 MB 49.2 MB/s eta 0:00:00


In [35]:
import cpi

In [40]:
cpi.update()
cpi_series = cpi.series
cpi_series =cpi_series.get()

In [41]:
df = cpi_series.to_dataframe()

In [45]:
df.query('period_type == "monthly" and year >= 2010') \
.loc[:,["date", "value"]] \
.set_index('date') \
.head(12)

,value
date,
2010-01-01,216.687
2010-02-01,216.741
2010-03-01,217.631
2010-04-01,218.009
2010-05-01,218.178
2010-06-01,217.965
2010-07-01,218.011
2010-08-01,218.312
2010-09-01,218.439
